# Motion correction + Online CNMF/CNMF_E code

Complete pipeline for online processing using CaImAn Online, but with motion correction and CNMF/CNMF_E separate instead of done simultaneously.

This allows to visualize if motion correction was well performed prior to start CNMF_E. If parameters are set and recordings are stable, one can use just OnACID

In [1]:
import logging
try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import numpy as np
import os

import caiman as cm
from caiman.source_extraction import cnmf
import matplotlib.pyplot as plt

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11172\4207315497.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_11172\4207315497.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


# First import data and set parameters for motion_correction

Complete pipeline for online processing using CaImAn Online, but with motion correction and CNMF/CNMF_E separate instead of done simultaneously.

This allows to visualize if motion correction was well performed prior to start CNMF_E. If parameters are set and recordings are stable, one can use just OnACID

In [2]:
#Set data as a list
fnames = ["Z:/Shared/For_Rui/Data_20230214/12937-06AI_1DPI_S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06AI_3DPI_S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06AI_6DPI_S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06AI2_1DPI_S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06AI2_3DPI_S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06AI2_6DPI_S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06BI_01S.avi",
         "Z:/Shared/For_Rui/Data_20230214/12937-06BI_02S.avi"]

if isinstance(fnames,str):
    fnames = [fnames]

    ##############################################################################################################
#Set parameters
gSig_filt = None                    #Gaussian filter size. It should normally be half the neuron size. Set it to none for 2-photon
pw_rigid = True                     #True if piece-wise motion correction should be used
strides = (96, 96)                  #Patch sizes in pixels.
overlaps = (24, 24)                 #Overlap between patches in pixels. This value normally does not have to be very high.
max_shifts_online = 40              #Maximum rigid shifts
max_deviation_rigid = 10            #Maximum deviation in pixels between rigid shifts and shifts of individual patches
border_nan = 'copy'                 #flag for allowing NaN in the boundaries. True allows NaN, whereas 'copy' copies the 
                                            #value of the nearest data point.
shifts_opencv = True                #flag for applying shifts using cubic interpolation (otherwise FFT)
init_batch = 500                    #initial batch 
ds_factor = 2                       #Spatial binning factor. Have in account that if spatial binning is used, all values above are unaltered.
normalize = True                    #Whether to normalize each frame prior to online processing. Set it to False for 2 photon

params_dict = {'fnames': fnames,
               'gSig_filt': gSig_filt,
               'pw_rigid': pw_rigid,
               'strides': strides,
               'overlaps': overlaps,
               'max_shifts_online': max_shifts_online,
               'max_deviation_rigid': max_deviation_rigid,
               'border_nan': border_nan,
               'shifts_opencv': shifts_opencv,
               'init_batch': init_batch,
               'ds_factor': ds_factor}
opts = cnmf.params.CNMFParams(params_dict=params_dict)

nbits = np.float16                          #Number of bits in which to save final images as tiff files.
init_template = None                #If an initial template can be provided, this should be where to set this variable into 
                                           #the template 2D numpy array.


        9380 [params.py:                 set():976] [11172] Changing key fnames in group data from None to ['Z:/Shared/For_Rui/Data_20230214/12937-06AI_1DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI_3DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI_6DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI2_1DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI2_3DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI2_6DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06BI_01S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06BI_02S.avi']
        9380 [params.py:                 set():976] [11172] Changing key max_shifts_online in group online from 10 to 40
        9381 [params.py:                 set():976] [11172] Changing key init_batch in group online from 200 to 500
        9382 [params.py:                 set():976] [11172] Changing key ds_factor in group online from 1 to 2
        9382 [params.py:                 set():976] [11172] Changing key pw_rigid in group 

# Motion correction

In [ ]:
cnm = cnmf.online_cnmf.OnACID(params=opts)

for i in range(0,len(fnames)):
    cnm.params.set('data',{'fnames':[fnames[i]]})
    cnm.motion_correction_online(template = init_template, save_movie=True,nbits=nbits)

    fnames[i] = os.path.splitext(fnames[i])[0]+str('_MC.tif')

       13067 [params.py:                 set():976] [11172] Changing key fnames in group data from ['Z:/Shared/For_Rui/Data_20230214/12937-06AI_1DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI_3DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI_6DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI2_1DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI2_3DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06AI2_6DPI_S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06BI_01S.avi', 'Z:/Shared/For_Rui/Data_20230214/12937-06BI_02S.avi'] to ['Z:/Shared/For_Rui/Data_20230214/12937-06AI_1DPI_S.avi']
       13269 [online_cnmf.py:motion_correction_online():944] [11172] Analyzing Z:/Shared/For_Rui/Data_20230214/12937-06AI_1DPI_S.avi


# Set up parameters for CNMF or CNMF_E

Online calcium analysis can be done in patches. However that is normally not necessary.
In this case, patches are not being used.
Moreover, the preprocessing of images is removed in this example.

For a good online analysis, it is important to define a good initialization. That depends on some variables such as:

<ul>
    <li><b>init_batch: </b> It says the number of frames used for initializing the spatial components. Decrease if the kernel dies due to lack of memory.</li>
    <li><b>K: </b> Number of components to find in initialization. If set to None, the code tries to find based on a matemathical calculus as if running the normal CNMF pipeline.</li>
    <li><b>gSig: </b> Average radius of neurons. This value should be set to None in 2-photon data where neurons are too small.</li>
    <li><b>gSiz: </b> Half size of neuron bounding box. This value does not need to be initialized unless when one wants to detect non round shapes, such as dendrites.</li>
    <li><b>center_psf: Set true for 1 photon data. Set false for 2 photon data </b></li>
    <li><b>method_init: </b> Method for detect initialspatial components. <b>Should be set as 'corr_pnr' for 1 photon and 'greedy_roi' for 2 photon. </b></li>
    <li><b>init_method: </b> Method for initialize components. It can be 'bare', 'seeded' or 'cnmf'. </li>
</ul>

In [ ]:
#INITIALIZATION VARIABLES
####################################################################################################################################################
init_batch = 3000                                                   # number of frames to initialize components
K = 2                                                               # initial number of components
center_psf = True                                                   # wther to use 1 photon or 2 photon analysis
ssub = 2                                                            # satial subsampling
ssub_B = 2                                                          # downsampling factor for background during corr_pnr
tsub = 1                                                            # temporal subsampling
nb = 1                                                              # number of background components. Set to 2 if background fluctuates a lot.
method_init = 'corr_pnr'                                            # Method for initialize spatial components. Use 'greedy_roi' for 2-photon and 'corr_pnr'for 1 photon
init_method = 'cnmf'                                                # initialization method. It can be 'bare', 'cnmf' or 'seeded'
ring_size_factor = 1.5                                              # Radius of ring (*gSig) for computing backgroud corr_pnr

#SPATIAL COMPONENTS VARIABLES
####################################################################################################################################################
gSig = (4,4)                                                        # expected half size of neurons
#gSiz = (9,9)                                                       # half-size of bounding box for each neuron. This value is by default 2*gSig + 1
min_corr = 0.8                                                      # minimum correlation value for accepting spatial footprint in corr_pnr method
min_pnr = 0.7                                                       # minimum peak to noise ratio value for accepting spatial footprint in corr_pnr method
method_exp = 'dilate'                                               # method for expanding footprint of spatial components: 'ellipse' or 'dilate'
dist = 3                                                            # expansion factor of ellipse

#TEMPORAL COMPONENTS VARIABLES
####################################################################################################################################################
fr = 15                                                             # frame rate (Hz)
decay_time = 0.4                                                    # approximate length of transient event in seconds
p = 1                                                               # order of AR indicator dynamics
s_min = None                                                        # Minimum spike threshold amplitude (computed in the code if used). If negative is based on the absolute value

#ONLINE PROCESS VARIABLES
####################################################################################################################################################
expected_comps = 500                                                # total expected components
min_SNR = 1                                                         # minimum SNR for accepting new components
rval_thr = 0.90                                                     # correlation threshold for new component inclusion
#sniper_mode = True                                                 # flag using a CNN to detect new neurons (o/w space correlation is used)
expected_comps = 500                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
dist_shape_update = True                                            # flag for updating shapes in a distributed way
min_num_trial = 10                                                  # number of candidate components per frame     
epochs = 2                                                          # number of passes over the data
show_movie = False                                                  # show the movie with the results as the data gets processed
stop_detection = False                                              # stop detection on the last epoch

params_dict = {'fnames': fnames,
               'fr': fr,
               'decay_time': decay_time,
               'gSig': gSig,
               'p': p,
               'min_SNR': min_SNR,
               'rval_thr': rval_thr,
               'ds_factor': ds_factor,
               'nb': gnb,
               'motion_correct': False
               'init_batch': init_batch,
               'init_method': 'bare',
               'normalize': True,
               'expected_comps': expected_comps,
               'sniper_mode': sniper_mode,
               'dist_shape_update' : dist_shape_update,
               'min_num_trial': min_num_trial,
               'K': K,
               'epochs': epochs,
               'max_shifts_online': max_shifts_online,
               'pw_rigid': pw_rigid,
               'show_movie': show_movie}
opts = cnmf.params.CNMFParams(params_dict=params_dict)